In [65]:
import cPickle
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
#from datetime 
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import operator
#import pandas.io.data
from pandas_datareader import data, wb
import pandas_datareader as pdr
from sklearn.qda import QDA
import re
from dateutil import parser
from bt import Strategy#, Portfolio
import quandl

In [56]:
def getStock(symbol, start, end):
    """
    Downloads Stock from Yahoo Finance.
    Computes daily Returns based on Adj Close.
    Returns pandas dataframe.
    """
    
    df =  pdr.get_data_yahoo(symbol, start, end)

    df.columns.values[-1] = 'AdjClose'
    df.columns = df.columns + '_' + symbol
    df['Return_%s' %symbol] = df['AdjClose_%s' %symbol].pct_change()
    
#    print(df)
    
    return df

#d1 = datetime.datetime(1971,2,5)
#d2 = datetime.datetime(2014,9,19)

#df = getStock("^IXIC", d1, d2)

#df.columns

In [68]:
def getStockFromQuandl(symbol, name, start, end):
    """
    Downloads Stock from Quandl.
    Computes daily Returns based on Adj Close.
    Returns pandas dataframe.
    """
    import quandl
    df =  quandl.get(symbol, trim_start = start, trim_end = end, authtoken="your token")

    df.columns.values[-1] = 'AdjClose'
    df.columns = df.columns + '_' + name
    df['Return_%s' %name] = df['AdjClose_%s' %name].pct_change()
    
    return df

In [72]:
#df[df.columns[0]]
#df.iloc[:,[0,1,2]]
#df.iloc[Open_^IXIC]
#df.columns
#df['AdjClose_^IXIC']

#df.columns[0]

#del df

def getStockDataFromWeb(start_string, end_string):
    """
    Collects predictors data from Yahoo Finance and Quandl.
    Returns a list of dataframes.
    """
#    start = parser.parse(start_string)
    start = start_string
    end = end_string
#    end = parser.parse(end_string)
    
    nasdaq = getStock('^IXIC', start, end)
    frankfurt = getStock('^GDAXI', start, end)
    london = getStock('^FTSE', start, end)
    paris = getStock('^FCHI', start, end)
    hkong = getStock('^HSI', start, end)
    nikkei = getStock('^N225', start, end)
    australia = getStock('^AXJO', start, end)
    
#    djia = getStockFromQuandl("YAHOO/INDEX_DJI", 'Djia', start_string, end_string) 
    
#    out =  pd.io.data.get_data_yahoo(fout, start, end)
#    out.columns.values[-1] = 'AdjClose'
#    out.columns = out.columns + '_Out'
#    out['Return_Out'] = out['AdjClose_Out'].pct_change()
    
    return [nasdaq, frankfurt, london, paris, hkong, nikkei, australia]

d1 = datetime.datetime(1971,2,5)
d2 = datetime.datetime(2014,9,19)

#df = getStock

bigDF = getStockDataFromWeb(d1, d2)

In [12]:
def addFeatures(dataframe, adjclose, returns, n):
    """
    operates on two columns of dataframe:
    - n >= 2
    - given Return_* computes the return of day i respect to day i-n. 
    - given AdjClose_* computes its moving average on n days

    """
    
    return_n = adjclose[9:] + "Time" + str(n)
    dataframe[return_n] = dataframe[adjclose].pct_change(n)
    
    roll_n = returns[7:] + "RolMean" + str(n)
    dataframe[roll_n] = pd.rolling_mean(dataframe[returns], n)

#resume here...    
#addFeatures(bigDF, )

In [76]:
def applyRollMeanDelayedReturns(datasets, delta):
    """
    applies rolling mean and delayed returns to each dataframe in the list
    """
    for dataset in datasets:
        columns = dataset.columns    
        adjclose = columns[-2]
        returns = columns[-1]
        for n in delta:
            addFeatures(dataset, adjclose, returns, n)
    
    return datasets    

delta = {0, 1}

h = applyRollMeanDelayedReturns(bigDF, delta)

/Users/Anton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=0,center=False).mean()
/Users/Anton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=1,center=False).mean()


In [78]:
h[0]

,Open_^IXIC,High_^IXIC,Low_^IXIC,Close_^IXIC,Volume_^IXIC,AdjClose_^IXIC,Return_^IXIC,^IXICTime0,^IXICRolMean0,^IXICTime1,^IXICRolMean1
Date,,,,,,,,,,,
1971-02-05,100.000000,100.000000,100.000000,100.000000,0,100.000000,NaN,0.0,NaN,NaN,NaN
1971-02-08,100.839996,100.839996,100.839996,100.839996,0,100.839996,0.008400,0.0,NaN,0.008400,0.008400
1971-02-09,100.760002,100.760002,100.760002,100.760002,0,100.760002,-0.000793,0.0,NaN,-0.000793,-0.000793
1971-02-10,100.690002,100.690002,100.690002,100.690002,0,100.690002,-0.000695,0.0,NaN,-0.000695,-0.000695
1971-02-11,101.449997,101.449997,101.449997,101.449997,0,101.449997,0.007548,0.0,NaN,0.007548,0.007548
1971-02-12,102.050003,102.050003,102.050003,102.050003,0,102.050003,0.005914,0.0,NaN,0.005914,0.005914
1971-02-16,102.190002,102.190002,102.190002,102.190002,0,102.190002,0.001372,0.0,NaN,0.001372,0.001372
1971-02-17,101.739998,101.739998,101.739998,101.739998,0,101.739998,-0.004404,0.0,NaN,-0.004404,-0.004404
1971-02-18,101.419998,101.419998,101.419998,101.419998,0,101.419998,-0.003145,0.0,NaN,-0.003145,-0.003145


In [85]:
def mergeDataframes(datasets, index, cut):
    """
    merges datasets in the list 
    """
    subset = []#tion
    subset = [dataset.iloc[:, index:] for dataset in datasets[1:]]
    
    first = subset[0].join(subset[1:], how = 'outer')
    finance = datasets[0].iloc[:, index:].join(first, how = 'left') 
    finance = finance[finance.index > cut]
    return finance

index = 6

cut = datetime.datetime(1993, 1, 1)

mergedDF = mergeDataframes(bigDF, index, cut)

mergedDF

,Return_^IXIC,^IXICTime0,^IXICRolMean0,^IXICTime1,^IXICRolMean1,Return_^GDAXI,^GDAXITime0,^GDAXIRolMean0,^GDAXITime1,^GDAXIRolMean1,...,Return_^N225,^N225Time0,^N225RolMean0,^N225Time1,^N225RolMean1,Return_^AXJO,^AXJOTime0,^AXJORolMean0,^AXJOTime1,^AXJORolMean1
Date,,,,,,,,,,,,,,,,,,,,,
1993-01-04,-0.007608,0.0,NaN,-0.007608,-0.007608,-0.008931,0.0,NaN,-0.008931,-0.008931,...,0.004077,0.0,NaN,0.004077,0.004077,0.007286,0.0,NaN,0.007286,0.007286
1993-01-05,0.003781,0.0,NaN,0.003781,0.003781,0.016391,0.0,NaN,0.016391,0.016391,...,-0.008885,0.0,NaN,-0.008885,-0.008885,0.002411,0.0,NaN,0.002411,0.002411
1993-01-06,0.011137,0.0,NaN,0.011137,0.011137,0.000000,0.0,NaN,0.000000,0.000000,...,-0.003562,0.0,NaN,-0.003562,-0.003562,-0.015887,0.0,NaN,-0.015887,-0.015887
1993-01-07,-0.005338,0.0,NaN,-0.005338,-0.005338,-0.008931,0.0,NaN,-0.008931,-0.008931,...,-0.000119,0.0,NaN,-0.000119,-0.000119,-0.008168,0.0,NaN,-0.008168,-0.008168
1993-01-08,-0.001474,0.0,NaN,-0.001474,-0.001474,-0.007131,0.0,NaN,-0.007131,-0.007131,...,-0.008700,0.0,NaN,-0.008700,-0.008700,-0.020556,0.0,NaN,-0.020556,-0.020556
1993-01-11,0.007664,0.0,NaN,0.007664,0.007664,0.000326,0.0,NaN,0.000326,0.000326,...,-0.002705,0.0,NaN,-0.002705,-0.002705,0.008739,0.0,NaN,0.008739,0.008739
1993-01-12,-0.004323,0.0,NaN,-0.004323,-0.004323,-0.001175,0.0,NaN,-0.001175,-0.001175,...,0.005485,0.0,NaN,0.005485,0.005485,-0.003741,0.0,NaN,-0.003741,-0.003741
1993-01-13,0.010788,0.0,NaN,0.010788,0.010788,-0.008953,0.0,NaN,-0.008953,-0.008953,...,-0.009772,0.0,NaN,-0.009772,-0.009772,-0.010343,0.0,NaN,-0.010343,-0.010343
1993-01-14,0.012988,0.0,NaN,0.012988,0.012988,0.004748,0.0,NaN,0.004748,0.004748,...,-0.000121,0.0,NaN,-0.000121,-0.000121,0.008787,0.0,NaN,0.008787,0.008787


In [87]:
def applyTimeLag(dataset, lags, delta):
    """
    apply time lag to return columns selected according  to delta.
    Days to lag are contained in the lads list passed as argument.
    Returns a NaN free dataset obtained cutting the lagged dataset
    at head and tail
    """
    
#dataset.Return_Out = dataset.Return_Out.shift(-1)
    maxLag = max(lags)

    columns = dataset.columns[::(2*max(delta)-1)]
    for column in columns:
        for lag in lags:
            newcolumn = column + str(lag)
            dataset[newcolumn] = dataset[column].shift(lag)

    return dataset.iloc[maxLag:-1,:]

lags = {0, 1, 2}

hhh = applyTimeLag(mergedDF, lags, delta)

hhh

,Return_^IXIC,^IXICTime0,^IXICRolMean0,^IXICTime1,^IXICRolMean1,Return_^GDAXI,^GDAXITime0,^GDAXIRolMean0,^GDAXITime1,^GDAXIRolMean1,...,^AXJOTime02,^AXJORolMean00,^AXJORolMean01,^AXJORolMean02,^AXJOTime10,^AXJOTime11,^AXJOTime12,^AXJORolMean10,^AXJORolMean11,^AXJORolMean12
Date,,,,,,,,,,,,,,,,,,,,,
1993-01-06,0.011137,0.0,NaN,0.011137,0.011137,0.000000,0.0,NaN,0.000000,0.000000,...,0.0,NaN,NaN,NaN,-0.015887,0.002411,0.007286,-0.015887,0.002411,0.007286
1993-01-07,-0.005338,0.0,NaN,-0.005338,-0.005338,-0.008931,0.0,NaN,-0.008931,-0.008931,...,0.0,NaN,NaN,NaN,-0.008168,-0.015887,0.002411,-0.008168,-0.015887,0.002411
1993-01-08,-0.001474,0.0,NaN,-0.001474,-0.001474,-0.007131,0.0,NaN,-0.007131,-0.007131,...,0.0,NaN,NaN,NaN,-0.020556,-0.008168,-0.015887,-0.020556,-0.008168,-0.015887
1993-01-11,0.007664,0.0,NaN,0.007664,0.007664,0.000326,0.0,NaN,0.000326,0.000326,...,0.0,NaN,NaN,NaN,0.008739,-0.020556,-0.008168,0.008739,-0.020556,-0.008168
1993-01-12,-0.004323,0.0,NaN,-0.004323,-0.004323,-0.001175,0.0,NaN,-0.001175,-0.001175,...,0.0,NaN,NaN,NaN,-0.003741,0.008739,-0.020556,-0.003741,0.008739,-0.020556
1993-01-13,0.010788,0.0,NaN,0.010788,0.010788,-0.008953,0.0,NaN,-0.008953,-0.008953,...,0.0,NaN,NaN,NaN,-0.010343,-0.003741,0.008739,-0.010343,-0.003741,0.008739
1993-01-14,0.012988,0.0,NaN,0.012988,0.012988,0.004748,0.0,NaN,0.004748,0.004748,...,0.0,NaN,NaN,NaN,0.008787,-0.010343,-0.003741,0.008787,-0.010343,-0.003741
1993-01-15,0.002084,0.0,NaN,0.002084,0.002084,0.013717,0.0,NaN,0.013717,0.013717,...,0.0,NaN,NaN,NaN,0.012340,0.008787,-0.010343,0.012340,0.008787,-0.010343
1993-01-18,0.001406,0.0,NaN,0.001406,0.001406,0.018451,0.0,NaN,0.018451,0.018451,...,0.0,NaN,NaN,NaN,0.002738,0.012340,0.008787,0.002738,0.012340,0.008787


In [56]:
def prepareDataForClassification(dataset, start_test):
    """
    generates categorical output column, attach to dataframe 
    label the categories and split into train and test
    """
    le = preprocessing.LabelEncoder()
    
    dataset['UpDown'] = dataset['Return_Out']
    dataset.UpDown[dataset.UpDown >= 0] = 'Up'
    dataset.UpDown[dataset.UpDown < 0] = 'Down'
    dataset.UpDown = le.fit(dataset.UpDown).transform(dataset.UpDown)
    
    features = dataset.columns[1:-1]
    X = dataset[features]    
    y = dataset.UpDown    
    
    X_train = X[X.index < start_test]
    y_train = y[y.index < start_test]              
    
    X_test = X[X.index >= start_test]    
    y_test = y[y.index >= start_test]
    
    return X_train, y_train, X_test, y_test   

In [57]:
def performClassification(X_train, y_train, X_test, y_test, method, parameters, fout, savemodel):
    """
    performs classification on daily returns using several algorithms (method).
    method --> string algorithm
    parameters --> list of parameters passed to the classifier (if any)
    fout --> string with name of stock to be predicted
    savemodel --> boolean. If TRUE saves the model to pickle file
    """
   
    if method == 'RF':   
        return performRFClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel)
        
    elif method == 'KNN':
        return performKNNClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel)
    
    elif method == 'SVM':   
        return performSVMClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel)
    
    elif method == 'ADA':
        return performAdaBoostClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel)
    
    elif method == 'GTB': 
        return performGTBClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel)

    elif method == 'QDA': 
        return performQDAClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel)

In [58]:
def performRFClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel):
    """
    Random Forest Binary Classification
    """
    clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
    clf.fit(X_train, y_train)
    
    if savemodel == True:
        fname_out = '{}-{}.pickle'.format(fout, datetime.now())
        with open(fname_out, 'wb') as f:
            cPickle.dump(clf, f, -1)    
    
    accuracy = clf.score(X_test, y_test)
    
    return accuracy

In [59]:
def performKNNClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel):
    """
    KNN binary Classification
    """
    clf = neighbors.KNeighborsClassifier()
    clf.fit(X_train, y_train)

    if savemodel == True:
        fname_out = '{}-{}.pickle'.format(fout, datetime.now())
        with open(fname_out, 'wb') as f:
            cPickle.dump(clf, f, -1)    
    
    accuracy = clf.score(X_test, y_test)
    
    return accuracy

In [60]:
def performSVMClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel):
    """
    SVM binary Classification
    """
    c = parameters[0]
    g =  parameters[1]
    clf = SVC()
    clf.fit(X_train, y_train)

    if savemodel == True:
        fname_out = '{}-{}.pickle'.format(fout, datetime.now())
        with open(fname_out, 'wb') as f:
            cPickle.dump(clf, f, -1)    
    
    accuracy = clf.score(X_test, y_test)
    
    return accuracy

In [61]:
def performAdaBoostClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel):
    """
    Ada Boosting binary Classification
    """
    n = parameters[0]
    l =  parameters[1]
    clf = AdaBoostClassifier()
    clf.fit(X_train, y_train)

    if savemodel == True:
        fname_out = '{}-{}.pickle'.format(fout, datetime.now())
        with open(fname_out, 'wb') as f:
            cPickle.dump(clf, f, -1)    
    
    accuracy = clf.score(X_test, y_test)
    
    return accuracy

In [62]:
def performGTBClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel):
    """
    Gradient Tree Boosting binary Classification
    """
    clf = GradientBoostingClassifier(n_estimators=100)
    clf.fit(X_train, y_train)

    if savemodel == True:
        fname_out = '{}-{}.pickle'.format(fout, datetime.now())
        with open(fname_out, 'wb') as f:
            cPickle.dump(clf, f, -1)    
    
    accuracy = clf.score(X_test, y_test)
    
    return accuracy

In [63]:
def performQDAClass(X_train, y_train, X_test, y_test, parameters, fout, savemodel):
    """
    Quadratic Discriminant Analysis binary Classification
    """
    def replaceTiny(x):
        if (abs(x) < 0.0001):
            x = 0.0001
    
    X_train = X_train.apply(replaceTiny)
    X_test = X_test.apply(replaceTiny)
    
    clf = QDA()
    clf.fit(X_train, y_train)

    if savemodel == True:
        fname_out = '{}-{}.pickle'.format(fout, datetime.now())
        with open(fname_out, 'wb') as f:
            cPickle.dump(clf, f, -1)    
    
    accuracy = clf.score(X_test, y_test)
    
    return accuracy

In [64]:
def performFeatureSelection(maxdeltas, maxlags, fout, cut, start_test, path_datasets, savemodel, method, folds, parameters):
    """
    Performs Feature selection for a specific algorithm
    """
    
    for maxlag in range(3, maxlags + 2):
        lags = range(2, maxlag) 
        print ''
        print '============================================================='
        print 'Maximum time lag applied', max(lags)
        print ''
        for maxdelta in range(3, maxdeltas + 2):
            datasets = loadDatasets(path_datasets, fout)
            delta = range(2, maxdelta) 
            print 'Delta days accounted: ', max(delta)
            datasets = applyRollMeanDelayedReturns(datasets, delta)
            finance = mergeDataframes(datasets, 6, cut)
            print 'Size of data frame: ', finance.shape
            print 'Number of NaN after merging: ', count_missing(finance)
            finance = finance.interpolate(method='linear')
            print 'Number of NaN after time interpolation: ', count_missing(finance)
            finance = finance.fillna(finance.mean())
            print 'Number of NaN after mean interpolation: ', count_missing(finance)    
            finance = applyTimeLag(finance, lags, delta)
            print 'Number of NaN after temporal shifting: ', count_missing(finance)
            print 'Size of data frame after feature creation: ', finance.shape
            X_train, y_train, X_test, y_test  = prepareDataForClassification(finance, start_test)
            
            print performCV(X_train, y_train, folds, method, parameters, fout, savemodel)
            print ''

NameError: name 'path_datasets' is not defined